# Create pseudo data to stimulate different fMRI parcellations

In [1]:
# Assuming there are 500000 voxels in a fMRI scan


# Pseudo data is created by sci-kit learn make_classification function

import numpy as np
import cupy as cp
import pandas as pd
import time
import pickle
from joblib import Parallel, delayed

from cuml import make_classification
# from sklearn.datasets import make_classification


# Get k parcels(patches) with same number of voxels(features)
def get_parcels(X, y, k, dtype='cp'):
    """
    X: numpy or cupy array, shape = (n_samples, n_features)
    y: numpy or cupy array, shape = (n_samples, )
    k: int, number of patches
    """
    # get number of voxels in each parcel
    n_voxels_per_parcel = int(X.shape[1]/k)
    # get k parcels
    parcels = []
    if dtype == 'cp':
        for i in range(k):
            parcels.append((X[:, i*n_voxels_per_parcel:(i+1)*n_voxels_per_parcel], y))
    else:
        for i in range(k):
            parcels.append((X[:, i*n_voxels_per_parcel:(i+1)*n_voxels_per_parcel].get(), y.get()))
        
    return parcels
    
    
    

# Get k parcels(patches) with random number of voxels(features) in each parcel
def get_parcels_diff(X, y, k=300, least_voxels_per_parcel=100, dtype='cp'):
    """
    X: numpy or cupy array, shape = (n_samples, n_features)
    y: numpy or cupy array, shape = (n_samples, )
    k: int, number of parcels
    least_voxels_per_parcel: int, the least number of voxels in each parcel
    """
    # get k parcels
    parcels = []
    if dtype == 'cp':
        for i in range(k):
            # get number of voxels in each parcel
            n_voxels_per_parcel = cp.random.randint(least_voxels_per_parcel, X.shape[1]//k)
            parcels.append((X[:, i*n_voxels_per_parcel:(i+1)*n_voxels_per_parcel], y))
    else:
        for i in range(k):
            # get number of voxels in each parcel
            n_voxels_per_parcel = np.random.randint(least_voxels_per_parcel, X.shape[1]//k)
            parcels.append((X[:, i*n_voxels_per_parcel:(i+1)*n_voxels_per_parcel].get(), y.get()))




# Compare the performance of sklearn and cuML (No parallelization)

In [2]:
# Compare the performance of sklearn and cuML by evaluating the accuracy of the model and 
# the time it takes to train the model of different parcellations

# hyperparameters
N_SAMPLES = 500 # 500 or 3000
N_INFORMATIVE_RATIO = 0.01 # 1% of voxels are informative
N_CLASSES = 8
DATA_TYPE = np.float32 # set data type to float32 to leaverage GPU
N_FEATURES = 300000
N_PATCHES = 300 # 20 or 300
SKLEARN_N_JOBS = 1 # number of jobs for sklearn
PARCEL_VOXELS = 'same' # 'same' or 'diff'


X, y = make_classification(n_samples=N_SAMPLES, 
                            n_features=N_FEATURES, 
                            n_informative=int(N_FEATURES*N_INFORMATIVE_RATIO),
                            n_classes=N_CLASSES)


# Create a list of different parcellations
parcellations = [get_parcels(X, y, N_PATCHES)]
parcellations_name = {}

# Performance log
df = pd.DataFrame(columns=['parcel_type', 'n_features', 'n_informative', 'n_classes', 'n_samples', 'sklearn_time', 'sklearn_accuracy', 'cuml_time', 'cuml_accuracy'])



### Sklearn

In [3]:
%%time
# Train and evaluate with sklearn with cross validation
from sklearn.ensemble import RandomForestClassifier as skRandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from cuml.metrics import accuracy_score

from tqdm import tqdm

for i in range(len(parcellations)):
    for j in tqdm(range(len(parcellations[i]))):
        # sklearn
        start = time.time()
        sk_model = skRandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=SKLEARN_N_JOBS)
        scores = cross_validate(sk_model, parcellations[i][j][0].get(), parcellations[i][j][1].get(), cv=5, scoring='accuracy')
        # scores = cross_validate(sk_model, parcellations[i][j][0], parcellations[i][j][1], cv=5, scoring='accuracy')
        end = time.time()
        sklearn_time = end - start
        sklearn_accuracy = scores['test_score'].mean()

        
        # log
        df = pd.concat([df, pd.DataFrame.from_records([{'parcel_type': 'sklearn_non_parallel',
                                            'n_features': parcellations[i][j][0].shape[1],
                                            'n_informative': N_INFORMATIVE_RATIO*parcellations[i][j][0].shape[1],
                                            'n_classes': N_CLASSES,
                                            'n_samples': N_SAMPLES,
                                            'sklearn_time': sklearn_time,
                                            'sklearn_accuracy': sklearn_accuracy,
                                            }])], ignore_index=True)
        



100%|██████████| 300/300 [03:29<00:00,  1.43it/s]

CPU times: user 3min 27s, sys: 1.45 s, total: 3min 28s
Wall time: 3min 29s


In [4]:
# # Save the log
# df.to_csv('performance_log_no_parallel.csv', index=False)

# Compare the performance of sklearn and cuML (Parallelization)

### cuML

In [5]:
# %%time
# # Train and evaluate with cuML with cross validation
# from cuml.ensemble import RandomForestClassifier as cuRandomForestClassifier
# from sklearn.model_selection import KFold

# kfold = KFold(n_splits=5)

# for i in range(len(parcellations)):
#     for j in tqdm(range(len(parcellations[i]))):

#         # cuML
#         fold_accuracy = cp.array([])
#         # X, y = cp.array(parcellations[i][j][0]), cp.array(parcellations[i][j][1])
#         start = time.time()
#         for train_idx, test_idx in kfold.split(X=parcellations[i][j][0], y=parcellations[i][j][1]):
#             cu_model = cuRandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
#             cu_model.fit(parcellations[i][j][0][train_idx], parcellations[i][j][1][train_idx])
#             # cu_model.fit(X[train_idx], y[train_idx])
#             fold_accuracy = cp.append(fold_accuracy, accuracy_score(parcellations[i][j][1][test_idx], cu_model.predict(parcellations[i][j][0][test_idx])))
#         end = time.time()
#         cuml_time = end - start
#         cuml_accuracy = cp.asnumpy(fold_accuracy).mean()
        
#         # log
#         df = pd.concat([df, pd.DataFrame.from_records([{'parcel_type': 'cuML',
#                             'n_features': parcellations[i][j][0].shape[1],
#                             'n_informative': N_INFORMATIVE_RATIO*parcellations[i][j][0].shape[1],
#                             'n_classes': N_CLASSES,
#                             'n_samples': N_SAMPLES,
#                             'cuml_time': cuml_time,
#                             'cuml_accuracy': cuml_accuracy,
#                             }])], ignore_index=True)

### Sklearn (Parallelization)

In [6]:
%%time
# parallelize the training and evaluation process for sklearn

BATCH_SIZE = 5 # number of models to train at a time
N_JOBS = 4 # number of CPUs to use for a RandomForestClassifier


# Define a function to train a model and return its score
def train_and_log(i, j, parcellation):
    # sklearn
    start = time.time()
    sk_model = skRandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=N_JOBS)
    scores = cross_validate(sk_model, parcellation[0].get(), parcellation[1].get(), cv=5, scoring='accuracy')
    # scores = cross_validate(sk_model, parcellation[0], parcellation[1], cv=5, scoring='accuracy')
    end = time.time()
    sklearn_time = end - start
    sklearn_accuracy = scores['test_score'].mean()

    # log
    return {'parcel_type': 'sklearn_parallel',
            'n_features': parcellation[0].shape[1],
            'n_informative': N_INFORMATIVE_RATIO*parcellation[0].shape[1],
            'n_classes': N_CLASSES,
            'n_samples': N_SAMPLES,
            'sklearn_time': sklearn_time,
            'sklearn_accuracy': sklearn_accuracy,
            }

# Create a list to store the results
results = []

# Define the number of models to train at a time as the number of CPUs
batch_size = BATCH_SIZE

# Flatten the list of parcellations and associated indices
flat_parcellations = [(i, j, parcellations[i][j]) for i in range(len(parcellations)) for j in range(len(parcellations[i]))]

# Use joblib to train models and log results in batches
for i in range(0, len(flat_parcellations), batch_size):
    batch_parcellations = flat_parcellations[i:i+batch_size]
    start = time.time()
    batch_results = Parallel(n_jobs=-1)(delayed(train_and_log)(*parcellation) for parcellation in batch_parcellations)
    end = time.time()
    
    average_results = {'parcel_type': 'sklearn_parallel',
            'n_features': parcellations[0][0][0].shape[1],
            'n_informative': N_INFORMATIVE_RATIO*parcellations[0][0][0].shape[1],
            'n_classes': N_CLASSES,
            'n_samples': N_SAMPLES,
            'sklearn_time': (end - start),
            'sklearn_accuracy': np.mean([result['sklearn_accuracy'] for result in batch_results])}
            
    
    results.extend([average_results])

# Convert results to DataFrame
df = pd.concat([df, pd.DataFrame.from_records(results)], ignore_index=True)
df

CPU times: user 729 ms, sys: 1 s, total: 1.73 s
Wall time: 43.4 s


,parcel_type,n_features,n_informative,n_classes,n_samples,sklearn_time,sklearn_accuracy,cuml_time,cuml_accuracy
0,sklearn_non_parallel,1000,10.0,8,500,0.760647,0.1440,NaN,NaN
1,sklearn_non_parallel,1000,10.0,8,500,0.707530,0.1420,NaN,NaN
2,sklearn_non_parallel,1000,10.0,8,500,0.693067,0.1580,NaN,NaN
3,sklearn_non_parallel,1000,10.0,8,500,0.696927,0.1520,NaN,NaN
4,sklearn_non_parallel,1000,10.0,8,500,0.702187,0.1480,NaN,NaN
...,...,...,...,...,...,...,...,...,...
355,sklearn_parallel,1000,10.0,8,500,0.734852,0.1524,NaN,NaN
356,sklearn_parallel,1000,10.0,8,500,0.726679,0.1528,NaN,NaN
357,sklearn_parallel,1000,10.0,8,500,0.741448,0.1480,NaN,NaN
358,sklearn_parallel,1000,10.0,8,500,0.725690,0.1408,NaN,NaN


In [7]:
# Save the log
# df.to_csv(f'result_nsapmles{N_SAMPLES}_npaches{N_PATCHES}_parcelsVoxels_{PARCEL_VOXELS}.csv', index=False)
df.to_csv(f'n_job1_result_nsapmles{N_SAMPLES}_npaches{N_PATCHES}_parcelsVoxels_{PARCEL_VOXELS}.csv', index=False)

### cuML (Parallelization)

In [8]:
# %%time
# # parallelize the training and evaluation process for cuML

# BATCH_SIZE = 5 # number of models to train at a time
# N_JOBS = 1 # number of CPUs to use for a RandomForestClassifier


# # Define a function to train a model and return its score
# def train_and_log(i, j, parcellation):
#     # cuML
#     fold_accuracy = cp.array([])
#     # start = time.time()
#     # for train_idx, test_idx in kfold.split(X=parcellation[0], y=parcellation[1]):
#     #     cu_model = cuRandomForestClassifier(n_estimators=100, max_depth=2)
#     #     cu_model.fit(parcellation[0][train_idx], parcellation[1][train_idx])
#     #     fold_accuracy = cp.append(fold_accuracy, accuracy_score(parcellation[1][test_idx], cu_model.predict(parcellation[0][test_idx])))
#     # end = time.time()
#     start = time.time()
#     cu_model = cuRandomForestClassifier(n_estimators=100, max_depth=2)
#     scores = cross_validate(cu_model, parcellation[0].get(), parcellation[1].get(), cv=5, scoring='accuracy')
#     end = time.time()
#     cuml_time = end - start
#     cuml_accuracy = cp.asnumpy(fold_accuracy).mean()

#     # log
#     return {'parcel_type': i,
#             'n_features': parcellation[0].shape[1],
#             'n_informative': N_INFORMATIVE_RATIO*parcellation[0].shape[1],
#             'n_classes': N_CLASSES,
#             'n_samples': N_SAMPLES,
#             'cuml_time': cuml_time,
#             'cuml_accuracy': cuml_accuracy,
#             }
    
    
# # Create a list to store the results
# results = []

# # Define the number of models to train at a time as the number of CPUs
# batch_size = BATCH_SIZE

# # Flatten the list of parcellations and associated indices
# flat_parcellations = [(i, j, parcellations[i][j]) for i in range(len(parcellations)) for j in range(len(parcellations[i]))]

# # Use joblib to train models and log results in batches
# for i in range(0, len(flat_parcellations), batch_size):
#     batch_parcellations = flat_parcellations[i:i+batch_size]
#     batch_results = Parallel(n_jobs=-1)(delayed(train_and_log)(*parcellation) for parcellation in batch_parcellations)
#     results.extend(batch_results)

# # Convert results to DataFrame
# df = pd.DataFrame(results)
